
# 1. Enunciado

Ahora que ya conocemos los fundamentos del web scraping, vamos a aplicarlos sobre un caso real:

1. Elige la web deportiva que desees que contenga datos de resultados, participantes, información de contexto del evento, etc.

2. Genera un script en Python que mediante web scraping extraiga la información de dicha web y al menos 5 subwebs, y genere un fichero de al menos 50 registros con información relativa a partidos, jugadores, etc.  
   Puedes scrapear todas las subwebs dentro de la principal que desees siempre que cumplas el mínimo de 50 registros y 5 subwebs.

3. El formato del fichero es libre.

---

# 2. Detalles de la entrega

- Desarrolla un script que cumpla todos los pasos del enunciado:
  - a. Si creas un notebook de Jupyter, debes convertirlo a HTML y subir dicho fichero a la plataforma, además del notebook en sí mismo.
  - b. Si usas un IDE o directamente desarrollas sobre un editor, debes subir el script al campus virtual.

- Sube el fichero generado con toda la información extraída de la web.

- Documenta todo el código explicando por qué has hecho cada cosa.

- Genera un documento donde se explique con secciones y un índice el desarrollo que has realizado.


# Primeros pasos en el proyecto de scraping de LaLiga

Este script representa la **primera prueba del proyecto**, donde el objetivo era familiarizarse con la extracción de datos de **resultados de partidos** de LaLiga utilizando Selenium y BeautifulSoup.

## Objetivos de esta primera prueba

1. **Navegar automáticamente** a la página de resultados de LaLiga.
2. **Esperar a que carguen los elementos dinámicos** de la página (tablas y filas de partidos).
3. **Extraer información básica** de cada partido:
   - Fecha y hora del partido
   - Equipos (local y visitante)
   - Resultado (goles de cada equipo)
4. **Almacenar los datos en una lista de diccionarios** para su posterior análisis o almacenamiento.

## Tecnologías y librerías utilizadas

- **Selenium**: Para automatizar la navegación y esperar a que los elementos de la página carguen dinámicamente.
- **BeautifulSoup**: Para parsear el HTML y seleccionar las filas y celdas de la tabla.
- **webdriver-manager**: Para gestionar automáticamente la instalación de ChromeDriver.
- **Python**: Como lenguaje principal del proyecto.

## Descripción del flujo

1. Se inicializa el navegador Chrome en modo **headless** para que funcione sin abrir la ventana.
2. Se accede a la URL de resultados de LaLiga.
3. Se espera dinámicamente hasta que aparezca una **tabla** con los resultados (max 20 segundos).
4. Se parsea el HTML con BeautifulSoup y se filtran todas las filas de partidos.
5. Se ignoran las filas que contienen información adicional (`row-more-info`) para centrarse solo en los partidos principales.
6. Para cada fila de partido se extraen:
   - Fecha y hora
   - Nombre del equipo local y visitante
   - Resultado de goles
7. Se guardan los datos en un diccionario y se añaden a la lista `partidos`.
8. Finalmente, se cierra el navegador y se imprimen los resultados.

## Aprendizajes de esta primera prueba

- Aprendí a combinar **Selenium y BeautifulSoup** para trabajar con páginas web dinámicas.
- Comprendí la necesidad de **esperas dinámicas** para que los elementos carguen correctamente antes de intentar extraerlos.
- Se sentaron las bases para desarrollar un **scraper más robusto** que posteriormente incluiría paginación, manejo de errores y extracción de estadísticas completas.

> Esta primera prueba sirvió como **piedra angular** del proyecto completo, permitiendo iterar y escalar el scraper hacia la extracción de estadísticas más complejas, como goleadores, asistencias y tarjetas.


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

URL = "https://www.laliga.com/laliga-easports/resultados"

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

partidos = []

try:
    driver.get(URL)

    # Esperamos a que cargue la tabla de resultados
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Todas las filas de partidos (ignoramos las row-more-info)
    filas = soup.select("tr.styled__TableRow-sc-43wy8s-4")

    for fila in filas:
        if "row-more-info" in fila.get("class", []):
            continue

        # Fecha y hora
        fecha = fila.select_one('td[type="date"] p')
        hora = fila.select_one('td[type="time"] p')

        # Equipos
        equipos = fila.select("p.jTXYPl")
        if len(equipos) != 2:
            continue

        equipo_local = equipos[0].get_text(strip=True)
        equipo_visitante = equipos[1].get_text(strip=True)

        # Resultado
        goles = fila.select("div.live p")
        if len(goles) >= 3:
            goles_local = goles[0].get_text(strip=True)
            goles_visitante = goles[2].get_text(strip=True)
        else:
            goles_local = goles_visitante = None

        partidos.append({
            "fecha": fecha.get_text(strip=True) if fecha else None,
            "hora": hora.get_text(strip=True) if hora else None,
            "local": equipo_local,
            "visitante": equipo_visitante,
            "goles_local": goles_local,
            "goles_visitante": goles_visitante
        })

finally:
    driver.quit()

# Mostrar resultados
for p in partidos:
    print(p)


{'fecha': 'VIE 23.01.2026', 'hora': '21:00', 'local': 'Levante UD', 'visitante': 'Elche CF', 'goles_local': '3', 'goles_visitante': '2'}
{'fecha': 'SAB 24.01.2026', 'hora': '14:00', 'local': 'Rayo Vallecano', 'visitante': 'CA Osasuna', 'goles_local': '1', 'goles_visitante': '3'}
{'fecha': 'SAB 24.01.2026', 'hora': '16:15', 'local': 'Valencia CF', 'visitante': 'RCD Espanyol de Barcelona', 'goles_local': '3', 'goles_visitante': '2'}
{'fecha': 'SAB 24.01.2026', 'hora': '18:30', 'local': 'Sevilla FC', 'visitante': 'Athletic Club', 'goles_local': '2', 'goles_visitante': '1'}
{'fecha': 'SAB 24.01.2026', 'hora': '21:00', 'local': 'Villarreal CF', 'visitante': 'Real Madrid', 'goles_local': '0', 'goles_visitante': '2'}
{'fecha': 'DOM 25.01.2026', 'hora': '14:00', 'local': 'Atlético de Madrid', 'visitante': 'RCD Mallorca', 'goles_local': '3', 'goles_visitante': '0'}
{'fecha': 'DOM 25.01.2026', 'hora': '16:15', 'local': 'FC Barcelona', 'visitante': 'Real Oviedo', 'goles_local': '3', 'goles_visita

# Scraping completo de resultados por jornada de LaLiga 2025-26

Este script representa la **segunda fase del proyecto**, donde se implementa un scraper más completo capaz de recorrer **todas las jornadas de la temporada 2025-26**, almacenando los resultados de todos los partidos en un archivo JSON.

## Objetivos principales

1. Navegar automáticamente a cada jornada de la temporada usando un **formato dinámico de URL**.
2. Detectar cuando una jornada **no existe o no tiene datos** para detener el scraping.
3. Extraer información de cada partido:
   - Fecha y hora
   - Equipo local y visitante
   - Resultado del partido (goles de cada equipo)
   - Jornada y temporada
4. Guardar todos los datos en un **archivo JSON** para análisis posterior.
5. Implementar **delays y control de errores** para no sobrecargar el servidor y manejar posibles páginas vacías.

## Tecnologías y librerías utilizadas

- **Selenium**: Para cargar páginas dinámicas y esperar elementos que se generan con JavaScript.
- **BeautifulSoup**: Para parsear el HTML y extraer la información de los partidos.
- **webdriver-manager**: Para instalar y gestionar automáticamente ChromeDriver.
- **Python (json y time)**: Para almacenar los resultados en archivos estructurados y manejar retrasos entre solicitudes.

## Descripción del flujo

1. Se inicializa el navegador en **modo headless**, permitiendo que el script corra en segundo plano sin abrir ventanas.
2. Se define la URL base de resultados de LaLiga, con un **placeholder para la jornada**.
3. Se inicia un bucle `while` que recorre cada jornada:
   - Se construye la URL de la jornada actual.
   - Se navega a la página y se espera hasta que cargue la tabla de resultados (máximo 15 segundos).
   - Si no aparece la tabla, se asume que no hay más jornadas y se termina el scraping.
4. Se parsea el HTML con BeautifulSoup y se filtran todas las filas de partidos relevantes.
5. Para cada fila válida, se extraen:
   - Fecha y hora del partido
   - Equipos (local y visitante)
   - Goles de cada equipo
   - Número de jornada y temporada
6. Se añade cada partido a la lista `todos_los_partidos`.
7. Se implementa un **retardo de 1 segundo** entre jornadas para evitar bloqueos por parte del servidor.
8. El bucle termina automáticamente cuando no hay partidos en la jornada o no se encuentra la tabla.
9. Finalmente, se guardan todos los resultados en un archivo JSON con indentación legible.

## Aprendizajes y mejoras respecto a la primera prueba

- Introducción de **paginación automática**, recorriendo todas las jornadas sin necesidad de especificar URLs manualmente.
- Manejo de **condiciones de fin de scraping**, evitando errores si una jornada no tiene datos.
- Uso de **JSON** para almacenar datos de manera estructurada y reutilizable.
- Implementación de **pausas controladas**, asegurando que el scraper sea amigable con el servidor web.
- Consolidación de los conocimientos previos en Selenium y BeautifulSoup para construir un scraper **robusto y escalable**.

> Este desarrollo marca un **gran avance** respecto a la primera prueba, ya que permite recorrer toda la temporada automáticamente, sirviendo como base para luego extraer estadísticas más complejas como goleadores, asistencias o tarjetas.


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import json
import time

BASE_URL = "https://www.laliga.com/laliga-easports/resultados/2025-26/jornada-{}"

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

todos_los_partidos = []

jornada = 1

try:
    while True:
        url = BASE_URL.format(jornada)
        print(f"🔍 Scrapeando jornada {jornada} → {url}")
        driver.get(url)

        try:
            # Esperamos a que cargue la tabla de resultados
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.TAG_NAME, "table"))
            )
        except:
            print(f"⛔ No hay datos para la jornada {jornada}. Fin del scraping.")
            break

        soup = BeautifulSoup(driver.page_source, "html.parser")

        filas = soup.select("tr.styled__TableRow-sc-43wy8s-4")

        partidos_jornada = 0

        for fila in filas:
            if "row-more-info" in fila.get("class", []):
                continue

            # Fecha y hora
            fecha = fila.select_one('td[type="date"] p')
            hora = fila.select_one('td[type="time"] p')

            # Equipos
            equipos = fila.select("p.jTXYPl")
            if len(equipos) != 2:
                continue

            local = equipos[0].get_text(strip=True)
            visitante = equipos[1].get_text(strip=True)

            # Resultado
            goles = fila.select("div.live p")
            if len(goles) >= 3:
                goles_local = goles[0].get_text(strip=True)
                goles_visitante = goles[2].get_text(strip=True)
            else:
                goles_local = goles_visitante = None

            todos_los_partidos.append({
                "temporada": "2025-26",
                "jornada": jornada,
                "fecha": fecha.get_text(strip=True) if fecha else None,
                "hora": hora.get_text(strip=True) if hora else None,
                "equipo_local": local,
                "equipo_visitante": visitante,
                "goles_local": goles_local,
                "goles_visitante": goles_visitante
            })

            partidos_jornada += 1

        if partidos_jornada == 0:
            print(f"⛔ Jornada {jornada} sin partidos. Fin.")
            break

        jornada += 1
        time.sleep(1)  # pequeño delay para no ir como un bot agresivo

finally:
    driver.quit()

# --- Guardar JSON ---
with open("resultados_laliga_2025_26.json", "w", encoding="utf-8") as f:
    json.dump(todos_los_partidos, f, ensure_ascii=False, indent=2)

print(f"\n✅ Scraping terminado. Jornadas guardadas: {jornada - 1}")
print(f"📦 Total partidos: {len(todos_los_partidos)}")


🔍 Scrapeando jornada 1 → https://www.laliga.com/laliga-easports/resultados/2025-26/jornada-1
🔍 Scrapeando jornada 2 → https://www.laliga.com/laliga-easports/resultados/2025-26/jornada-2
🔍 Scrapeando jornada 3 → https://www.laliga.com/laliga-easports/resultados/2025-26/jornada-3
🔍 Scrapeando jornada 4 → https://www.laliga.com/laliga-easports/resultados/2025-26/jornada-4
🔍 Scrapeando jornada 5 → https://www.laliga.com/laliga-easports/resultados/2025-26/jornada-5
🔍 Scrapeando jornada 6 → https://www.laliga.com/laliga-easports/resultados/2025-26/jornada-6
🔍 Scrapeando jornada 7 → https://www.laliga.com/laliga-easports/resultados/2025-26/jornada-7
🔍 Scrapeando jornada 8 → https://www.laliga.com/laliga-easports/resultados/2025-26/jornada-8
🔍 Scrapeando jornada 9 → https://www.laliga.com/laliga-easports/resultados/2025-26/jornada-9
🔍 Scrapeando jornada 10 → https://www.laliga.com/laliga-easports/resultados/2025-26/jornada-10
🔍 Scrapeando jornada 11 → https://www.laliga.com/laliga-easports/res

## Scraping de estadísticas dinámicas de LaLiga

En esta fase final del proyecto se desarrolló un scraper robusto para **obtener diferentes estadísticas de jugadores y equipos** de LaLiga, incluyendo:

- Goleadores
- Asistencias
- Tarjetas amarillas
- Tarjetas rojas
- Pases
- Paradas

El objetivo principal fue **automatizar la extracción de todas las páginas disponibles** para cada estadística, evitando introducir URLs manualmente.

---

### 1. Estructura del proyecto

- **PAGINAS**: Diccionario que asocia cada tipo de estadística con su archivo CSV de salida.
- **BASE_URL**: URL base con marcador para el tipo de estadística.
- **Selenium**: Configurado en modo headless para una ejecución eficiente.
- **Delays aleatorios**: Entre páginas y estadísticas, para simular navegación humana y evitar bloqueos.

---

### 2. Función `esperar_tabla`

Esta función permite:

- Esperar dinámicamente a que la tabla cargue filas `<tr>`.
- Detectar si la tabla está vacía o si no se cargó tras un tiempo determinado.
- Retornar la tabla lista para extraer datos o `None` si falla.

> Esto asegura robustez frente a páginas lentas o cambios en la carga de contenido.

---

### 3. Función `scrapear_estadistica`

El flujo de scraping por estadística es el siguiente:

1. Inicia en **página 1** y continúa hasta que no se encuentren filas.
2. Accede a la URL específica de la estadística y página.
3. Llama a `esperar_tabla` para asegurar que la tabla esté disponible.
4. Extrae **encabezados** solo en la primera página.
5. Recorre todas las filas de la tabla, extrayendo los datos de cada celda.
6. Acumula los datos en una lista `todas_las_filas`.
7. Aplica un **delay aleatorio entre páginas** para evitar detección como bot.
8. Guarda los datos en un archivo CSV al final del scraping de cada estadística.

> Este método permite recorrer automáticamente todas las páginas de cada estadística sin intervención manual.

---

### 4. Función `main`

- Recorre todos los tipos de estadísticas definidos en `PAGINAS`.
- Llama a `scrapear_estadistica` para cada tipo.
- Aplica un **delay aleatorio entre estadísticas**.
- Garantiza que Selenium se cierre correctamente al finalizar el scraping, incluso si ocurre un error.

---

### 5. Mejoras implementadas

- **Automatización completa** de URLs y paginación.
- **Control de errores** y condiciones de fin de scraping.
- **Robustez** frente a tablas vacías o retrasos en la carga de la página.
- **Simulación de navegación humana** mediante delays aleatorios.
- **Exportación estructurada a CSV**, lista para análisis estadístico.

> Con este enfoque, el proyecto es escalable para futuras temporadas o nuevas estadísticas, manteniendo un proceso automatizado y fiable.

---

### 6. Conclusión

La tercera fase consolidó un **scraper profesional y escalable** capaz de:

- Acceder a múltiples páginas por estadística.
- Extraer datos estructurados de tablas dinámicas.
- Guardar resultados en formatos reutilizables para análisis posteriores.

> Esta implementación asegura un flujo de trabajo reproducible y eficiente para el análisis completo de las estadísticas de LaLiga 2025-26.


In [4]:
import csv
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options

# Estadísticas y archivos
PAGINAS = {
    "goleadores": "goleadores.csv",
    "tarjetas-amarillas": "tarjetas_amarillas.csv",
    "tarjetas-rojas": "tarjetas_rojas.csv",
    "asistencias": "asistencias.csv",
    "pases": "pases.csv",
    "paradas": "paradas.csv"
}

BASE_URL = "https://www.laliga.com/estadisticas/laliga-easports/{}"

# Configuración de Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecutar en modo headless
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=chrome_options)

def esperar_tabla(driver, xpath_tabla, timeout=15):
    """Espera dinámicamente a que la tabla cargue filas <tr>"""
    try:
        print(f"⏳ Esperando la tabla: {xpath_tabla}")
        WebDriverWait(driver, timeout).until(
            EC.presence_of_all_elements_located((By.XPATH, xpath_tabla + "/tbody/tr"))
        )
        tabla = driver.find_element(By.XPATH, xpath_tabla)
        filas = tabla.find_elements(By.XPATH, ".//tbody/tr")
        if len(filas) == 0:
            print("⚠️ La tabla está vacía")
            return None
        print(f"✅ Tabla cargada con {len(filas)} filas")
        return tabla
    except TimeoutException:
        print(f"❌ La tabla no se cargó tras {timeout} segundos")
        return None

def scrapear_estadistica(nombre_estadistica):
    print(f"\n📊 Scrapeando {nombre_estadistica}...")

    pagina = 1
    todas_las_filas = []
    headers = None

    while True:
        url = BASE_URL.format(f"{nombre_estadistica}/pagina/{pagina}")
        print(f"🔗 Accediendo a {url}")
        driver.get(url)
        time.sleep(random.uniform(2, 4))  # Delay inicial

        xpath_tabla = '//*[@id="main-content"]/div/div[2]/div[2]/div/table'
        tabla = esperar_tabla(driver, xpath_tabla)

        if not tabla:
            print(f"🚫 No hay más datos en la página {pagina}. Terminando {nombre_estadistica}.")
            break

        # Extraer encabezados solo en la primera página
        if headers is None:
            headers = [th.text.strip() for th in tabla.find_elements(By.XPATH, ".//thead/tr/th")]
            print(f"🔍 Encabezados encontrados: {headers}")

        # Extraer filas
        for tr in tabla.find_elements(By.XPATH, ".//tbody/tr"):
            celdas = [td.text.strip() for td in tr.find_elements(By.XPATH, "./td")]
            todas_las_filas.append(celdas)
        print(f"🔹 Página {pagina} scrapeada. Filas totales hasta ahora: {len(todas_las_filas)}")

        pagina += 1
        time.sleep(random.uniform(2, 5))  # Delay entre páginas

    # Guardar CSV
    if headers and todas_las_filas:
        archivo_csv = PAGINAS[nombre_estadistica]
        with open(archivo_csv, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(headers)
            writer.writerows(todas_las_filas)
        print(f"💾 Datos guardados en {archivo_csv}")
    else:
        print(f"⚠️ No se encontraron datos para {nombre_estadistica}")

def main():
    for estadistica in PAGINAS.keys():
        scrapear_estadistica(estadistica)
        time.sleep(random.uniform(3, 6))  # Delay entre tipos de estadísticas

if __name__ == "__main__":
    try:
        main()
    finally:
        driver.quit()
        print("\n✅ Scraper finalizado")



📊 Scrapeando goleadores...
🔗 Accediendo a https://www.laliga.com/estadisticas/laliga-easports/goleadores/pagina/1
⏳ Esperando la tabla: //*[@id="main-content"]/div/div[2]/div[2]/div/table
✅ Tabla cargada con 20 filas
🔍 Encabezados encontrados: ['', '', 'NOMBRE', 'EQUIPO', 'GOLES A FAVOR', 'PARTIDOS JUGADOS', 'GOLES POR PARTIDO']
🔹 Página 1 scrapeada. Filas totales hasta ahora: 20
🔗 Accediendo a https://www.laliga.com/estadisticas/laliga-easports/goleadores/pagina/2
⏳ Esperando la tabla: //*[@id="main-content"]/div/div[2]/div[2]/div/table
✅ Tabla cargada con 20 filas
🔹 Página 2 scrapeada. Filas totales hasta ahora: 40
🔗 Accediendo a https://www.laliga.com/estadisticas/laliga-easports/goleadores/pagina/3
⏳ Esperando la tabla: //*[@id="main-content"]/div/div[2]/div[2]/div/table
✅ Tabla cargada con 20 filas
🔹 Página 3 scrapeada. Filas totales hasta ahora: 60
🔗 Accediendo a https://www.laliga.com/estadisticas/laliga-easports/goleadores/pagina/4
⏳ Esperando la tabla: //*[@id="main-content"]